Instalamos las librerías necesarias para el notebook

In [1]:
#!pip install spacy
#!pip install nltk
#!pip install sklearn

Importamos las librerias y funciones que utilizaremos

In [2]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

Leemos el archivo CSV para nuestro Bag of Words

In [3]:
df = pd.read_csv('df_mini_HS.csv')

Definimos una función que se utilizará para limpieza de los datos (eliminando menciones a otros usuarios, enlaces, puntos, comas, números y otros elementos que no nos son de utilidad para el Bag of Words).

Posterior, generamos una columna en la que se almacenará el texto limpio.

In [4]:
def limpiar(text):
  # Eliminar menciones a usuarios, enlaces, hashtags, números, paréntesis y corchetes, caracteres de nueva línea, puntos y comas,
  # signos de admiración, signos de exclamación
  text = re.sub(r"@\S+|http[s]?\://\S+|#\S+|[0-9]+|(\(.*\))|(\[.*\])|\n", "", text)
  return text

df['texto_limpio'] = df['text'].apply(limpiar)

Ahora definimos una función para tokenizar nuestro texto limpio. Este texto lo guardaremos en una columna **tokens**.

In [5]:
nltk.download('punkt')

def tokenizar(text):
  text = word_tokenize(text)
  return text

df['tokens'] = df['texto_limpio'].apply(tokenizar)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Genereraremos una función para filtrar *stopwords* de nuestro texto. Almacenamos las palabras filtradas en una nueva columna.

In [6]:
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')

# nuestra función filtra stopwords tanto en español como en inglés, ya que el dataset contiene oraciones en ambos idiomas
def filtrar(tokens):
  text = [palabra for palabra in tokens if palabra not in spanish_stopwords]
  text = [palabra for palabra in text if palabra not in english_stopwords]
  return text

df['palabras_filtradas'] = df['tokens'].apply(filtrar)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Se define una función para lematizar las oraciones y se almacenan en una nueva columna.

In [7]:
#!sudo python3 -m spacy download es

In [8]:
nlp = spc.load("es_core_news_sm")

def lematizar(text):
  text = nlp(" ".join(text))
  text = " ".join([token.lemma_ for token in text])
  return text

df['oracion_lematizada'] = df['palabras_filtradas'].apply(lematizar)

Guardamos nuestras oraciones lematizadas en un nuevo dataframe, el cual será la base para nuestra Bag of Words.

In [9]:
text_data = df['oracion_lematizada'].values

Importamos el vectorizador.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(text_data)

CountVectorizer()

Transformamos nuestros datos con el vectorizador.

In [11]:
vectors = vectorizer.transform(text_data)

In [12]:
vocabulary = vectorizer.get_feature_names_out()

In [13]:
df_bow = pd.DataFrame.sparse.from_spmatrix(vectors, columns=vocabulary)

Imprimimos nuestro Bag of Words para analizar que se ha realizado correctamente.

In [14]:
print(df_bow)

    acoso  agar  agresión  al  amiga  amodio  analicer  aprieto  arab  arar  \
0       0     0         1   0      0       0         0        1     0     0   
1       0     0         0   0      1       1         0        0     0     0   
2       0     0         0   0      0       0         0        0     0     0   
3       0     0         0   0      0       0         0        0     0     0   
4       0     0         0   0      0       0         0        0     0     0   
5       0     0         0   0      0       0         0        0     0     0   
6       0     0         0   0      0       0         0        0     0     0   
7       0     0         0   0      0       0         0        0     0     0   
8       0     0         0   0      0       0         0        0     0     0   
9       1     0         0   0      0       0         1        0     0     0   
10      0     0         0   0      0       0         0        0     1     1   
11      0     1         0   0      0       0        

Ahora podemos apreciar que algunas palabras no fueron filtradas de manera correcta. Las filtraremos de forma manual.

In [15]:
# Nos deshacemos de las columnas con stopwords y números.
columns_to_remove = ['el', 'él', 'si', 'que', 'qué', 'de', 'no', 'por', 'tres', 'al', 'vía', 'vez', 'así', 'ese', 'en', 'pero', 'otro','dia',
                     'porque']
df_bow_filtered = df_bow.drop(columns=columns_to_remove)

También combinaremos las columnas de "árabe", "árabes" y "arab", así como las de "callate" y "cállate".

In [16]:
# Creamos una nueva columna llamada "arabe" que contendrá nuestros valores anteriores.
df_bow_filtered['arabe'] = df_bow_filtered[['arab', 'árabe', 'árabes']].sum(axis=1)

# Hacemos drop a las columnas anteriores
df_bow_filtered = df_bow_filtered.drop(['arab', 'árabe', 'árabes'], axis=1)

# Combinamos "cállate" y "callate" a una sola columna y hacemos drop a la otra
df_bow_filtered['callate'] = df_bow_filtered[['callate', 'cállate']].sum(axis=1)
df_bow_filtered = df_bow_filtered.drop(['cállate'], axis=1)

Veremos que palabras se repiten más en los enunciados marcados con un label 1 y 0.

In [17]:
# Agrupamos los datos por su label para poder contar las palabras.
label_word_counts = df_bow_filtered.groupby(df['label']).sum()
word_count_difference = label_word_counts.loc[1] - label_word_counts.loc[0]
print(word_count_difference.sort_values(ascending=False))

callate        3
mantero        1
olvidar        1
expulsarlos    1
estado         1
              ..
equipo        -1
oler          -1
futbol        -2
mundo         -2
arabe         -2
Length: 166, dtype: Sparse[int64, 0]


Se puede apreciar que el Bag of Words contiene bastantes columnas, demasiadas como para poder realizar un análisis a simple vista..

---


Transformaremos los datos para mostrar solo las palabras que aparecen en más de una ocasión, de la siguiente manera:

In [18]:
# Se calcula la suma de cada columna.
col_sum = df_bow_filtered.sum(axis=0)

# Se crea una máscara booleana que cumpla la condición "la suma de la columna es mayor a 1".
mask = col_sum > 1

# Filtramos nuestro dataframe con nuestra máscara.
df_bow_2 = df_bow_filtered.loc[:, mask]
df_bow_2

,acoso,callate,decir,futbol,inmigrante,mundo,perra,provocación,puta,puto,arabe
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,1,0,0
5,0,1,0,0,0,0,0,0,0,1,0
6,0,0,0,0,1,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,0
9,1,0,1,0,0,0,0,1,0,0,0


Podemos ver en nuestra Bag of Words un preocupante dato sobre las oraciones analizadas: se repiten palabras como "inmigrante", "acoso", "arab" (se filtraron árabe y árabes al aparecer cada una solo en una oración, pero se pueden contar como apariciones adicionales de la palabra), junto a insultos y palabras algo más violentas. De aquí podemos ver que las oraciones, probablemente recabadas de redes sociales, implican cierto nivel de intolerancia y xenofobia, particularmente a la gente árabe.

De nuevo contaremos cuantas veces se repite cada palabra en los label 1 y 0, ahora con las palabras filtradas, para ver si podemos encontrar algún patrón entre los enunciados marcados con 1 y aquellos marcados con 0.

In [19]:
# Group the dataframe by the 'label' column and sum the word counts for each group
label_counts2 = df_bow_2.groupby(df['label']).sum()

# Calculate the difference in word counts between label 1 and label 0
word_count_difference = label_counts2.loc[1] - label_counts2.loc[0]

# Print the words with the largest positive difference (more frequent in label 1)
print("Words more frequent in label 1:\n", word_count_difference.sort_values(ascending=False))

Words more frequent in label 1:
 callate        3
perra          1
puta           1
acoso          0
decir          0
inmigrante     0
provocación    0
puto           0
futbol        -2
mundo         -2
arabe         -2
dtype: Sparse[int64, 0]


Podríamos decir que los enunciados con label 1 se pueden leer como ofensivos o agresivos, y los enunciados con label 0 pueden ser palabras más ordinarias o comunes.

---
Haremos de nuevo el Bag of Words, completo, ahora con la columna de característica indicado al final. Marcaremos label 1 como **agresivo** y label 0 como **ordinario**.


In [20]:
# Creamos la columna nueva "clase"
df_bow_filtered['clase'] = df['label'].map({1: 'agresivo', 0: 'ordinario'})
df_bow_filtered

,acoso,agar,agresión,amiga,amodio,analicer,aprieto,arar,ayuda,bala,...,unidos,user,vacación,verdad,vida,viola,voolka,yogurines,arabe,clase
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,agresivo
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,agresivo
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,agresivo
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,agresivo
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,agresivo
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,agresivo
6,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,agresivo
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,agresivo
8,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,agresivo
9,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,agresivo
